## Testing

In [98]:
from bigchaindb_driver import BigchainDB
from bigchaindb_driver.crypto import generate_keypair

import json

In [110]:
BDG_ROOT_URL = '35.157.161.130'
BDB_URL_PORT = 9984

In [111]:
bdb = BigchainDB('http://{}:{}'.format(BDG_ROOT_URL, BDB_URL_PORT))

In [112]:
alice = generate_keypair()

In [113]:
# Alice's bike
bicycle = {
    'data': {
        'bicycle': {
            'serial_number': 'abcd1234',
            'manufacturer': 'bkfab',
        },
    }, 
}

In [114]:
# Asset creation

# create asset
prepared_creation_tx = bdb.transactions.prepare(
        operation='CREATE',
        signers=alice.public_key,
        asset=bicycle,
)

fulfilled_creation_tx = bdb.transactions.fulfill(
    prepared_creation_tx, 
    private_keys=alice.private_key, # signed with private key
)

# send asset
sent_creation_tx = bdb.transactions.send(fulfilled_creation_tx)

In [116]:
txid = fulfilled_creation_tx['id']

In [117]:
txid

'368c05a467e89f529a32dcccbc265fab8447f953c3f450107a9231362fa044fc'

In [123]:
bdb.assets.get(search='abcd1234')

[{'data': {'bicycle': {'manufacturer': 'bkfab', 'serial_number': 'abcd1234'}},
  'id': '368c05a467e89f529a32dcccbc265fab8447f953c3f450107a9231362fa044fc'}]

## Clases

In [ ]:
def basemodel(database):
    class Model:
        db = database

    return Model

In [ ]:
class User(basemodel('database')):
    def __init__(self):
        self._public_key, self._private_key = generate_keypair()
        
    @staticmethod
    def from_dict(dct):
        self._private_key = dct.private_key
        self._public_key = dct.public_key
    
    @property
    def private_key(self):
        return self._private_key
    
    @property
    def public_key(self):
        return self._public_key

In [ ]:
class Asset(basemodel('database')):
    
    def __init__(self, owner: User, **kwargs):
        """
        Usage:
        Asset(me, id=3, value=4)
        ->
        {
            'data': {
                id : 3,
                value : 4,
            }
        }
        """
        self.asset = {
            'data': {
                k:v for k,v in kwargs.items()
            }
        }
        self.owner = owner
        self._txid = None
        
    @property
    def id(self):
        if self._id is None:
            raise Exception('The Asset must be created first')
        return self._id
        
    def create(self):
        bdb = Asset.db
        # PREPARE CREATION
        prepared_creation_tx = bdb.transactions.prepare(
            operation='CREATE',
            signers=self.user.public_key,
            asset=self.asset,
        )
        
        # FULFILL IT
        fulfilled_creation_tx = bdb.transactions.fulfill(
            prepared_creation_tx, 
            private_keys=self.private_key,
        )
        
        # WARNING: it might take a while until it is validated
        sent_creation_tx = bdb.transactions.send(fulfilled_creation_tx)
    
        self._id = fulfilled_creation_tx['id']
        
    def send(self, to: User):
        bdb = Asset.db
        assert(bdb.transactions.status(txid).get('status') == 'valid')
        
        creation_tx = bdb.transactions.retrieve(self.id)
        asset_id = creation_tx['id']
        
        transfer_asset = {
            'id': asset_id,
        }
        
        # PREPARE TRANSACTION
        output_index = 0

        output = creation_tx['outputs'][output_index
        transfer_input = {
            'fulfillment': output['condition']['details'],
            'fulfills': {
                 'output_index': output_index,
                 'transaction_id': creation_tx['id'],
             },
             'owners_before': output['public_keys'],
        }

        prepared_transfer_tx = bdb.transactions.prepare(
            operation='TRANSFER',
            asset=transfer_asset,
            inputs=transfer_input,
            recipients=to.public_key,
        )
        
        # FULFILL TRANSACTION
        fulfilled_transfer_tx = bdb.transactions.fulfill(
            prepared_transfer_tx,
            private_keys=self.user.private_key,
        )
        
        sent_transfer_tx = bdb.transactions.send(fulfilled_transfer_tx)

In [ ]:
class Voter(User):
    